In [1]:
%load_ext autoreload

In [2]:
%autoreload

In [3]:
import sys

In [4]:
from pathlib import Path
from kyoto_reader import KyotoReader
import json

In [5]:
import sys; sys.path.append('../src')
from data_loader.dataset.pas_dataset import PASDataset
import data_loader.dataset as module_dataset
from utils.parse_config import ConfigParser
from utils import read_json

In [6]:
# Tokenizer
from tokenizer import (
    BertTokenizeHandler,
    BartSPMTokenizeHandler,
    BiBartTokenizeHandler,
    T5TokenizeHandler,
    MBartTokenizerHandler,
    TokenizeHandlerMeta
)
MODEL2TOKENIZER = {
    'bert': BertTokenizeHandler,
    'bart': BartSPMTokenizeHandler,
    't5': T5TokenizeHandler,
    'mbart': MBartTokenizerHandler,
    'bibart': BiBartTokenizeHandler
}

In [7]:
CONFIG_PATH = "../config/BaselineBiBartModel-wn-8e-bibart-cz-vpa-npa-cr.json"
with open(CONFIG_PATH) as f:
    config = json.load(f)

In [8]:
print(config)

{'name': 'BaselineBiBartModel-wn-8e-bibart-cz-vpa-npa-cr', 'n_gpu': 8, 'arch': {'type': 'BaselineBiBartModel', 'args': {'pretrained_path': '/mnt/hinoki/umakoshi/ja_en_bart/huggingface_weights/v1', 'vocab_size': 64010, 'dropout': 0.0, 'num_case': 4, 'coreference': True}}, 'train_datasets': {'kwdlc': {'type': 'PASDataset', 'args': {'path': '/mnt/elm/umakoshi/transformers_pas_analysis/data/bibart_kwdlc_kc/kwdlc/train', 'cases': ['ガ', 'ヲ', 'ニ', 'ガ２'], 'exophors': ['著者', '読者', '不特定:人', '不特定:物'], 'coreference': True, 'bridging': False, 'max_seq_length': 170, 'model_name': 'bibart', 'pretrained_path': '/mnt/hinoki/umakoshi/ja_en_bart/huggingface_weights/v1', 'training': True, 'kc': False, 'train_targets': ['case', 'zero'], 'pas_targets': ['pred', 'noun']}}, 'kc': {'type': 'PASDataset', 'args': {'path': '/mnt/elm/umakoshi/transformers_pas_analysis/data/bibart_kwdlc_kc/kc_split/train', 'cases': ['ガ', 'ヲ', 'ニ', 'ガ２'], 'exophors': ['著者', '読者', '不特定:人', '不特定:物'], 'coreference': True, 'bridging': F

In [9]:
print(config["train_datasets"]["kwdlc"]["args"])

{'path': '/mnt/elm/umakoshi/transformers_pas_analysis/data/bibart_kwdlc_kc/kwdlc/train', 'cases': ['ガ', 'ヲ', 'ニ', 'ガ２'], 'exophors': ['著者', '読者', '不特定:人', '不特定:物'], 'coreference': True, 'bridging': False, 'max_seq_length': 170, 'model_name': 'bibart', 'pretrained_path': '/mnt/hinoki/umakoshi/ja_en_bart/huggingface_weights/v1', 'training': True, 'kc': False, 'train_targets': ['case', 'zero'], 'pas_targets': ['pred', 'noun']}


In [10]:
model_name = config["train_datasets"]["kwdlc"]["args"]["model_name"]
pretrained_path = config["train_datasets"]["kwdlc"]["args"]["pretrained_path"]
tokenizer_cls = MODEL2TOKENIZER[model_name]
tokenizer = tokenizer_cls.from_pretrained(pretrained_path, do_lower_case=False,
                                                       tokenize_chinese_chars=False)

In [11]:
reader = KyotoReader(Path(config["train_datasets"]["kc"]["args"]["path"]), extract_nes=False)

In [12]:
documents = reader.process_all_documents()

In [17]:
def tokenize_document(document, tokenizer):
    words = []
    for sentence in document:
        for bp in sentence.bps:
            for mrph in bp.mrph_list():
                words.append(mrph.midasi)
    before_joint_tokenized = [token for word in words for token in tokenizer.tokenize(word)]
    after_joint_tokenied = [token for token in tokenizer.tokenize(" ".join(words))]
    return words, before_joint_tokenized, after_joint_tokenied

In [22]:
SPM_SPECIAL_TOKEN = "▁"

def _count_special_token(words):
    """Count SPM_SPECIAL_TOKEN in words.
    """
    count = 0
    for word in words:
        if SPM_SPECIAL_TOKEN in word:
            count += 1
    return count

In [23]:
for document in documents:
    words, before, after = tokenize_document(document, tokenizer)
    if before != after:
        print("before != after")
        print(before)
        print(after)
        print()
    if _count_special_token(before) != len(words):
        print("count_special_token != len(words)", _count_special_token(before), len(words))
        print(before)
        print(words)
        print()

before != after
['▁うす', '▁の', '▁形', '▁は', '▁日本', '▁と', '▁同じ', '▁', '。', '▁', 'き', 'ね', '▁の', '▁形', '▁は', '▁月', '▁で', '▁ウサギ', '▁が', '▁使', 'って', '▁いる', '▁アノ', '▁', '長', 'い', '▁ヤツ', '▁だ', '▁', '。', '▁つく', '▁の', '▁は', '▁男', '▁の', '▁仕事', '▁', '。', '▁酒', '▁の', '▁回り', '▁も', '▁早く', '▁なる', '▁', '。', '▁夜', '▁も', '▁白', '々', 'と', '▁明け', 'る', '▁ころ', '▁、', '▁', '酔い', 'つ', 'ぶ', 'れた', '▁青年', '▁を', '▁母', '▁と', '▁', '妹', '▁が', '▁家', '▁に', '▁運ぶ', '▁', '。', '▁「', '▁もう', '▁一', '▁杯', '▁', '」', '▁「', '▁', 'て', 'や', 'んで', 'え', '▁', '」', '▁と', '▁', '気', '勢', '▁を', '▁上げ', 'て', '▁、', '▁飲み', '▁続け', 'る', '▁猛', '者', '▁も', '▁いる', '▁', '。', '▁国', '境', '▁', '貿易', '▁で', '▁暮ら', 'す', '▁小さ', 'な', '▁', '村', '▁', '。', '▁出', '稼', 'ぎ', '▁の', '▁若', 'い', '▁', '衆', '▁も', '▁', '都会', '▁から', '▁', '戻り', '▁、', '▁酒', '▁酒', '▁酒', '▁', '.', '.....', '▁で', '▁新', '年', '▁は', '▁始ま', 'る', '▁', '。']
['▁うす', '▁の', '▁形', '▁は', '▁日本', '▁と', '▁同じ', '▁', '。', '▁', 'き', 'ね', '▁の', '▁形', '▁は', '▁月', '▁で', '▁ウサギ', '▁が', '▁使', 'って', '▁いる', '▁アノ', '▁',

before != after
['▁「', '▁農', '業', '▁が', '▁でき', '▁なく', '▁なる', '▁ので', '▁は', '▁', '」', '▁と', '▁出', '稼', 'ぎ', '▁を', '▁', '加', '速', '▁さ', '▁せ', 'る', '▁', '。', '▁三', '▁世帯', '▁十二', '▁人', '▁の', '▁大', '▁家族', '▁の', '▁コー', 'イ', '▁さん', '▁', '宅', '▁で', '▁は', '▁、', '▁父', '▁と', '▁', '夫', '▁が', '▁台湾', '▁に', '▁', '。', '▁月', '▁二', '万', '▁', 'バーツ', '▁の', '▁', '仕', '送', 'り', '▁が', '▁届', 'く', '▁', '。', '▁出', '稼', 'ぎ', '▁世帯', '▁の', '▁大', '半', '▁は', '▁、', '▁テレビ', '▁や', '▁バイ', 'ク', '▁を', '▁買', 'う', '▁が', '▁、', '▁', '夫', '▁の', '▁', '手', '紙', '▁に', '▁は', '▁「', '▁仕事', '▁が', '▁', 'きつい', '▁', '。', '▁帰り', '▁たい', '▁', '」', '▁の', '▁文字', '▁', '。', '▁「', '▁野良', '▁仕事', '▁も', '▁', '力', '▁仕事', '▁も', '▁', '女', '▁だけ', '▁で', '▁は', '▁滞', 'る', '▁', '.', '.....', '▁', '。', '▁時代', '▁の', '▁流れ', '▁か', '▁', '」', '▁と', '▁ナイ', 'フェー', '▁', '村', '長', '▁', '。']
['▁「', '▁農', '業', '▁が', '▁でき', '▁なく', '▁なる', '▁ので', '▁は', '▁', '」', '▁と', '▁出', '稼', 'ぎ', '▁を', '▁', '加', '速', '▁さ', '▁せ', 'る', '▁', '。', '▁三', '▁世帯', '▁十二', '▁人', '▁の', '▁大', '▁

before != after
['▁変', '化', '▁の', '▁ない', '▁「', '▁男', '性', '▁', '」', '▁に', '▁', '愛想', 'づか', 'し', '▁...', '▁女性', '▁と', '▁共', '有', '▁', 'できる', '▁', '服', '▁へ', '▁「', '▁これ', 'ほど', '▁集ま', 'る', '▁と', '▁は', '▁', '.', '.....', '▁', '」', '▁', '。', '▁昨', '年', '▁十一', '▁月', '▁、', '▁東京', '▁・', '▁原', '宿', '▁で', '▁最終', '▁選', '考', '▁会', '▁の', '▁行', 'わ', '▁れた', '▁第', '▁七', '▁回', '▁「', '▁', 'ジュノ', 'ン', '▁・', '▁スーパー', '▁ボーイ', '▁コンテ', 'スト', '▁', '」', '▁', '。', '▁女性', '▁', '誌', '▁「', '▁', 'ジュノ', 'ン', '▁', '」', '▁が', '▁主', '催', '▁して', '▁「', '▁美', '男', '▁No', '▁', '1', '▁', '」', '▁を', '▁選', 'ぶ', '▁“', '▁逆', '▁ミス', 'コン', '▁”', '▁', '。', '▁', '関係', '▁者', '▁を', '▁驚', 'か', '▁せた', '▁の', '▁は', '▁、', '▁前', '回', '▁', 'まで', '▁二', '▁、', '▁三千', '▁人', '▁', '台', '▁だった', '▁', '応', '募', '▁者', '▁が', '▁、', '▁五千', '四', '百', '五', '十七', '▁人', '▁へ', '▁', 'ほぼ', '▁倍', '増', '▁した', '▁こと', '▁だった', '▁', '。']
['▁変', '化', '▁の', '▁ない', '▁「', '▁男', '性', '▁', '」', '▁に', '▁', '愛想', 'づか', 'し', '▁...', '▁女性', '▁と', '▁共', '有', '▁', 'できる', '▁', 

before != after
['▁不安', '▁', '点', '▁は', '▁、', '▁出', '足', '▁を', '▁止め', '▁られ', 'る', '▁と', '▁', '足', '▁が', '▁そろ', 'い', '▁が', 'ちな', '▁こと', '▁で', '▁、', '▁突き', '▁押し', '▁も', '▁腰', '▁が', '▁', '入', 'ら', '▁ず', '▁', '威力', '▁を', '▁失', 'う', '▁', '。', '▁けいこ', '▁でも', '▁', '曙', '▁や', '▁武蔵', '丸', '▁など', '▁、', '▁出', '足', '▁を', '▁受け止め', 'る', '▁', '力', '▁の', '▁ある', '▁', '力士', '▁が', '▁相手', '▁に', '▁なる', '▁と', '▁、', '▁この', '▁欠', '点', '▁が', '▁たび', 'たび', '▁顔', '▁を', '▁出', 'す', '▁', '。', '▁', '師', '匠', '▁の', '▁武蔵', '川', '▁', '親', '方', '▁も', '▁「', '▁', '足', '▁が', '▁止ま', 'る', '▁と', '▁、', '▁もう', '▁いけな', 'い', '▁', '。', '▁', '気', '▁を', '▁つけ', 'ろ', '▁と', '▁', '言って', '▁いる', '▁が', '▁', '.', '.....', '▁', '」', '▁と', '▁心配', 'げ', '▁', '。']
['▁不安', '▁', '点', '▁は', '▁、', '▁出', '足', '▁を', '▁止め', '▁られ', 'る', '▁と', '▁', '足', '▁が', '▁そろ', 'い', '▁が', 'ちな', '▁こと', '▁で', '▁、', '▁突き', '▁押し', '▁も', '▁腰', '▁が', '▁', '入', 'ら', '▁ず', '▁', '威力', '▁を', '▁失', 'う', '▁', '。', '▁けいこ', '▁でも', '▁', '曙', '▁や', '▁武蔵', '丸', '▁など', '▁、', '▁出', '足'

before != after
['▁と', 'ころが', '▁、', '▁この', '▁', '病院', '▁で', '▁通', '常', '▁、', '▁二', '▁人', '▁部', '屋', '▁を', '▁一', '▁人', '▁で', '▁使用', '▁する', '▁', '場', '合', '▁の', '▁', '料', '金', '▁は', '▁五千', '▁円', '▁で', '▁、', '▁一', '▁日', '▁一万', '▁円', '▁の', '▁「', '▁充', '電', '▁', '料', '▁', '」', '▁を', '▁計', '上', '▁して', '▁いた', '▁こと', '▁に', '▁なる', '▁', '。', '▁取', '材', '▁に', '▁対し', '▁、', '▁同', '▁', '病院', '▁の', '▁', '事務', '▁', '長', '▁は', '▁「', '▁', '携', '帯', '▁電', '話', '▁の', '▁こと', '▁は', '▁記', '録', '▁に', '▁なく', '▁、', '▁九', '▁月', '▁に', '▁辞', '任', '▁した', '▁前', '任', '▁者', '▁が', '▁管理', '▁して', '▁いた', '▁ので', '▁分か', 'ら', '▁ない', '▁が', '▁、', '▁特別', 'な', '▁介護', '▁の', '▁必要な', '▁病', '気', '▁で', '▁は', '▁ない', '▁し', '▁', '.', '.....', '▁', '。', '▁なぜ', '▁こんな', 'に', '▁多額', 'の', '▁', '室', '料', '▁差', '額', '▁に', '▁なった', '▁の', '▁か', '▁分か', 'ら', '▁ない', '▁', '」', '▁と', '▁困', '惑', '▁、', '▁', '調査', '▁し', '▁たい', '▁と', '▁して', '▁いる', '▁', '。']
['▁と', 'ころが', '▁、', '▁この', '▁', '病院', '▁で', '▁通', '常', '▁、', '▁二', '▁人', '▁部', '屋', '▁を', '▁一', '▁人

...

before != after
['▁今', '後', '▁、', '▁大', '▁事', '故', '▁に', '▁つなが', 'る', '▁恐れ', '▁も', '▁ある', '▁こと', '▁から', '▁、', '▁公', '団', '▁は', '▁この', '▁日', '▁、', '▁現', '場', '▁に', '▁監', '視', '▁カメラ', '▁を', '▁新', '設', '▁する', '▁など', '▁', '対策', '▁を', '▁始めた', '▁が', '▁、', '▁公', '団', '▁から', '▁水', '蒸気', '▁の', '▁飛', '散', '▁防', '止', '▁を', '▁要', '望', '▁さ', '▁れた', '▁', '工場', '▁側', '▁は', '▁「', '▁四十', '▁年', '▁以', '上', '▁も', '▁前', '▁から', '▁操', '業', '▁して', '▁おり', '▁、', '▁いま', 'さら', '▁移', '転', '▁も', '▁でき', '▁ない', '▁し', '▁', '.', '.....', '▁', '」', '▁と', '▁', '頭', '▁を', '▁抱え', 'て', '▁いる', '▁', '。']
['▁今', '後', '▁、', '▁大', '▁事', '故', '▁に', '▁つなが', 'る', '▁恐れ', '▁も', '▁ある', '▁こと', '▁から', '▁、', '▁公', '団', '▁は', '▁この', '▁日', '▁、', '▁現', '場', '▁に', '▁監', '視', '▁カメラ', '▁を', '▁新', '設', '▁する', '▁など', '▁', '対策', '▁を', '▁始めた', '▁が', '▁、', '▁公', '団', '▁から', '▁水', '蒸気', '▁の', '▁飛', '散', '▁防', '止', '▁を', '▁要', '望', '▁さ', '▁れた', '▁', '工場', '▁側', '▁は', '▁「', '▁四十', '▁年', '▁以', '上', '▁も', '▁前', '▁から', '▁操', '業', '▁して', '▁おり', '▁、', '▁いま

before != after
['▁しかし', '▁、', '▁', '高校', '▁に', '▁', '入', 'って', '▁から', '▁の', '▁時間', '▁は', '▁瞬く', '間', 'に', '▁流れ', '▁、', '▁', '気', '▁が', '▁つけ', 'ば', '▁その', '▁二十', '▁歳', '▁', '。', '▁年', '々', '▁、', '▁時', '▁が', '▁早く', '▁過ぎ', 'て', '▁いく', '▁ような', '▁', '気', '▁が', '▁する', '▁', '。', '▁そんな', '▁時', '▁の', '▁流れ', '▁に', '▁ついて', '▁いけ', '▁ない', '▁の', '▁か', '▁、', '▁今', '▁、', '▁自分', '▁を', '▁見つめ', 'る', '▁と', '▁年', '齢', '▁相応', 'の', '▁', '自覚', '▁を', '▁持て', '▁ず', '▁に', '▁いる', '▁の', '▁が', '▁、', '▁よく', '▁わかる', '▁', '。', '▁こんな', '▁こと', '▁を', '▁', '言う', '▁と', '▁、', '▁ここ', '▁数', '▁年', '▁言わ', '▁れて', '▁いる', '▁「', '▁三十', '▁歳', '▁', '成', '人', '▁説', '▁', '」', '▁や', '▁「', '▁', '成', '人', '▁', '式', '▁七五', '三', '▁', '」', '▁の', '▁ように', '▁、', '▁自', '立', '▁でき', 'て', '▁', 'い', '▁ない', '▁若', '者', '▁と', '▁たたか', '▁れ', '▁そうだ', '▁が', '▁', '.', '.....', '▁', '。']
['▁しかし', '▁、', '▁', '高校', '▁に', '▁', '入', 'って', '▁から', '▁の', '▁時間', '▁は', '▁瞬く', '間', 'に', '▁流れ', '▁、', '▁', '気', '▁が', '▁つけ', 'ば', '▁その', '▁二十', '▁歳', '▁', '。', '▁年', '々', 

before != after
['▁現', '在', '▁の', '▁社会', '党', '▁は', '▁自民', '党', '▁と', '▁の', '▁', '握', '手', '▁に', '▁よる', '▁村山', '▁富', '市', '▁首', '相', '▁の', '▁誕生', '▁', '以来', '▁、', '▁', '党', '▁内', '▁が', '▁', 'ギス', 'ギス', '▁して', '▁いた', '▁が', '▁、', '▁昨', '年', '▁末', '▁以', '降', '▁の', '▁早', '期', '▁新', '党', '▁結', '成', '▁派', '▁と', '▁、', '▁そう', '▁で', '▁ない', '▁グループ', '▁と', '▁の', '▁対', '立', '▁が', '▁鮮明', 'に', '▁なった', '▁', '。', '▁村山', '▁首', '相', '▁は', '▁、', '▁それ', '▁', 'まで', '▁の', '▁社会', '党', '▁の', '▁方', '針', '▁を', '▁「', '▁時代', '▁の', '▁変', '化', '▁に', '▁応じ', 'て', '▁', '.', '.....', '▁', '」', '▁と', '▁いう', '▁ような', '▁名', '▁の', '▁', '下', '▁に', '▁次', '々', '▁と', '▁放', '棄', '▁して', '▁、', '▁社会', '党', '▁', '委員', '▁', '長', '▁が', '▁首', '相', '▁を', '▁務め', 'て', '▁', 'いて', '▁も', '▁、', '▁実', '態', '▁は', '▁自民', '党', '▁と', '▁', '変わら', '▁ない', '▁形', '▁で', '▁連', '立', '▁政権', '▁を', '▁担', '当', '▁して', '▁いる', '▁の', '▁が', '▁実', '情', '▁である', '▁', '。']
['▁現', '在', '▁の', '▁社会', '党', '▁は', '▁自民', '党', '▁と', '▁の', '▁', '握', '手', '▁に', '▁よる', '▁村山', '

before != after
['▁寝', '室', '▁は', '▁', '布', '団', '▁を', '▁二', '▁つ', '▁敷', 'く', '▁だけ', '▁の', '▁スペース', '▁と', '▁', '布', '団', '▁を', '▁入れ', 'る', '▁クロ', 'ゼット', '▁と', '▁いう', '▁最小', '▁限', '▁に', '▁し', '▁、', '▁あと', '▁は', '▁', 'キ', 'チン', '▁と', '▁も', '▁続いた', '▁リビング', '▁一', '▁部', '屋', '▁に', '▁改', '築', '▁した', '▁', '。', '▁壁', '▁に', '▁沿', 'って', '▁奥行', 'き', '▁六十', '▁センチ', '▁の', '▁クロ', 'ゼット', '▁を', '▁', '造', 'り', '▁付け', '▁、', '▁衣', '類', '▁から', '▁本', '▁', 'まで', '▁を', '▁収', '納', '▁した', '▁', '。', '▁真ん', '中', '▁に', '▁は', '▁ガラス', '▁テーブル', '▁が', '▁リビング', '▁の', '▁“', '▁顔', '▁”', '▁と', '▁して', '▁、', '▁食事', '▁に', '▁も', '▁仕事', '▁に', '▁も', '▁', '.', '.....', '▁と', '▁、', '▁多目的', 'に', '▁活', '用', '▁さ', '▁れて', '▁いる', '▁', '。']
['▁寝', '室', '▁は', '▁', '布', '団', '▁を', '▁二', '▁つ', '▁敷', 'く', '▁だけ', '▁の', '▁スペース', '▁と', '▁', '布', '団', '▁を', '▁入れ', 'る', '▁クロ', 'ゼット', '▁と', '▁いう', '▁最小', '▁限', '▁に', '▁し', '▁、', '▁あと', '▁は', '▁', 'キ', 'チン', '▁と', '▁も', '▁続いた', '▁リビング', '▁一', '▁部', '屋', '▁に', '▁改', '築', '▁した', '▁', '。', '▁壁', '▁

before != after
['▁高田', '▁氏', '▁は', '▁幅広', 'い', '▁支', '援', '▁を', '▁得', 'る', '▁ため', '▁、', '▁八', '▁月', '▁半', 'ば', '▁に', '▁連', '合', '▁滋', '賀', '▁の', '▁東郷', '▁栄', '司', '▁会', '長', '▁に', '▁相', '談', '▁した', '▁', '。', '▁連', '合', '▁滋', '賀', '▁が', '▁推', 'す', '▁前', '▁民主', '▁改', '革', '▁連', '合', '▁', '代表', '▁の', '▁中村', '▁鋭', '一', '▁参', '院', '▁議', '員', '▁は', '▁', '衆', '院', '▁くら', '替え', '▁の', '▁意', '向', '▁', '。', '▁連', '合', '▁滋', '賀', '▁は', '▁参', '院', '▁の', '▁議', '席', '▁を', '▁死', '守', '▁する', '▁ため', '▁、', '▁高田', '▁氏', '▁に', '▁「', '▁連', '合', '▁公', '認', '▁候', '補', '▁なら', '▁支', '援', '▁する', '▁', '」', '▁と', '▁の', '▁条', '件', '▁で', '▁十', '▁月', '▁末', '▁に', '▁擁', '立', '▁を', '▁決めた', '▁', '。', '▁そこ', '▁で', '▁自民', '党', '▁県', '連', '▁は', '▁', '党', '▁本部', '▁から', '▁「', '▁連', '合', '▁と', '▁組', 'む', '▁なんて', '▁', '.', '.....', '▁', '」', '▁と', '▁も', '▁言わ', '▁れた', '▁と', '▁いう', '▁', '。']
['▁高田', '▁氏', '▁は', '▁幅広', 'い', '▁支', '援', '▁を', '▁得', 'る', '▁ため', '▁、', '▁八', '▁月', '▁半', 'ば', '▁に', '▁連', '合', '▁滋', '賀', '▁の', '▁東郷', '▁